In [ ]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [ ]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-13 23:48:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.50MB/s    in 0.2s    

2023-03-13 23:48:26 (4.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [ ]:
# Read in the data from an S3 Bucket
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-MM-dd")
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE|2015-08-31 00:00:00|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...| 

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Get the number of rows in the DataFrame.
print(f'number of rows in the Data Frame = {df.count()}')

number of rows in the Data Frame = 2302401


In [ ]:
# Drop null values, by default is any which means will drop any rows which has a NA
print((df.count(),len(df.columns)))
df2 = df.dropna()
df3 = df2.dropDuplicates()
print((df3.count(),len(df3.columns)))
df3.show()


(2302401, 15)
(2302174, 15)
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   15644771|R100GF4836CVA9|B0000E014E|     533431966|Terramar Women's ...|        Outdoors|          5|            0|          0|   N|                Y|Nice stuff; incor...|Fits well, keeps ...|2015-01-21 00:00:00|
|         US|   21774903|R1015BJNB5X8JC|B004P5XOTA|     5742

In [ ]:
df3.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

# Transform the Data

## Create the "review_id_table".

In [ ]:
# from pyspark.sql.functions import to_date
from pyspark.sql.functions import *
from pyspark.sql.types import *
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table_df = df3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_df2 = review_id_table_df.withColumn("review_date",to_date(col("review_date")))
# review_id_table_df.show(truncate=False)


In [ ]:
type(review_id_table_df2)

pyspark.sql.dataframe.DataFrame

In [ ]:
review_id_table_df2.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_table_df2.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100GF4836CVA9|   15644771|B0000E014E|     533431966| 2015-01-21|
|R1015BJNB5X8JC|   21774903|B004P5XOTA|     574224922| 2015-06-01|
|R101K7H9652YPL|   44641100|B00PLG6FZM|     300607595| 2015-08-25|
|R1026XJRFNZYY4|   45214968|B002QAVQ14|     558253757| 2014-08-20|
|R102FCGC30VU1M|    6814669|B00OP4MH62|     226080130| 2015-07-12|
|R102R2YX5PT1KE|   20168750|B00165N88O|     663045829| 2014-05-26|
|R102Y50B81GJV2|   45778690|B00LVP6ZZM|     502612531| 2014-12-22|
|R1030OA5DPCNZN|   52786988|B00QEC3XZC|     903076621| 2015-03-13|
|R103FLVTAYQLPZ|   10872475|B00JX5W56U|     382622505| 2014-07-16|
|R103IOF24MG7JN|   33045796|B0009KF4H0|     405897453| 2015-03-28|
|R103KLAGCQ9YLW|   12690557|B006GJ8Q7S|     483017301| 2015-04-07|
|R104O7NM2AYLGA|    5815764|B007Q4O3RY|     411742069| 2015-08

## Create the "products" Table

In [ ]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df2 = df3.select(["product_id", "product_title"])
products_df = products_df2.dropDuplicates()
print((products_df.count(),len(products_df.columns)))
products_df.show()

(391700, 2)
+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IFHFJXI|Ivation Portable ...|
|B00WG0J0D0|JanSport Superbre...|
|B00V15AUN0|Nickelodeon Paw P...|
|B00FUWSTI8|Bago Lightweight ...|
|B003FV94NA|Michelin Lithion ...|
|B00WIK04HO|Ultra Bright Camp...|
|B00J2HSCM0|High Sierra Tank ...|
|B009I6NSR4|Black Veil Brides...|
|B001GSHSLE|Stansport 191 App...|
|B00L2IO9M4|Columbia Sportswe...|
|B00KY7IM7W|Nalgene 32 Oz Wid...|
|B00TV5JCTK|Rollerblade ABEC ...|
|B00B9D071Y|BUFF UV Multifunc...|
|B00F9IGIKO|Condor Tactical F...|
|B004X55L9I|Hydro Flask Insul...|
|B00LORROIY|Scuba Choice Divi...|
|B00AATRU8G|Kelty Redwing 44 ...|
|B00HMCYWEO|Dakine Explorer L...|
|B004DK1CM8|Hot Headz 12V Hea...|
|B00T4W6SSS|Fits Sock Light H...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [ ]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 


from pyspark.sql.functions import desc,count

customers_df= df3.groupBy('customer_id').agg(count("customer_id").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30605925|             8|
|   19369914|             1|
|   45874295|             1|
|   31725909|             2|
|   45475563|             3|
|   12005663|             1|
|    3270869|             5|
|   49945790|             2|
|    6664524|             9|
|   21651140|             2|
|   12532704|             1|
|   19850820|            17|
|   10878516|             1|
|   47457340|             1|
|   20528213|             1|
|   10400847|             8|
|   30800260|             4|
|   11172320|             1|
|   11035586|             2|
|   23081895|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



## Create the "vine_table".

In [ ]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100GF4836CVA9|          5|            0|          0|   N|
|R1015BJNB5X8JC|          5|            0|          0|   N|
|R101K7H9652YPL|          5|            0|          0|   N|
|R1026XJRFNZYY4|          2|            0|          0|   N|
|R102FCGC30VU1M|          5|            0|          0|   N|
|R102R2YX5PT1KE|          2|            0|          0|   N|
|R102Y50B81GJV2|          5|           10|         11|   Y|
|R1030OA5DPCNZN|          5|            0|          0|   N|
|R103FLVTAYQLPZ|          5|            2|          4|   N|
|R103IOF24MG7JN|          3|            0|          0|   N|
|R103KLAGCQ9YLW|          1|            0|          0|   N|
|R104O7NM2AYLGA|          5|            0|          1|   N|
|R10503H9JLF6U9|          1|            0|          0|   N|
|R1055Z40PSYCIV|          2|           2

# Load

In [ ]:
# mode = "append"
# jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
# config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cailygz0tqjt.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", "password": "123postgres456", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_table_df2.write.jdbc(url=jdbc_url, table='review_id_table_Outdoors', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_Outdoors', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_Outdoors', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table_Outdoors', mode=mode, properties=config)